<font size="5"><b><center>CS:GO Professional Matches Analysis</center></b></font>
<center><img src="3.jpg" width="800" /></center>

# Introduction

<b>Background:</b> The data used in this notebook was collected at https://www.hltv.org/results. The data has been retreived from Kaggle.

Data Link: https://www.kaggle.com/datasets/mateusdmachado/csgo-professional-matches.

The original dataset contains the data of CSGO professional matches from 2016 to 2020, but I have extracted the data of professional matches happened between 2019 and 2020.

We will analyse the CSGO professional matches in each major between 2019 & 2020. The main focus is given on the rankings of the teams and individual players. The idea is just to explore the data and see what information I could get. Also, I tried to avoid computing statistics that are easily obtained in e-sports websites (team victory percentage, average k/d ratio per player, etc.). 

Let's get started!

# The Game

Counter-Strike: Global Offensive (CS:GO) is a 2012 multiplayer tactical first-person shooter developed by Valve and Hidden Path Entertainment. It is the fourth game in the Counter-Strike series.
Counter-Strike Global Offensive is a game released in 2012, as a sequel to Counter-Strike Source (released in 2004), which is itself a sequel to the original Counter-Strike (released in 2000). The game's longevity is primarily caused by its competitive approach and vibrant professional scene. This longevity has shown in numbers recently, as CS:GO reached in March its all-time high concurrent weekly players (1.1M players), making it the most played game on Steam.

The game retains the same gameplay concepts since its first version, which include a Terrorist side (T) that is tasked to plant a bomb and have it detonate, and a Counter-Terrorist side (CT) that is tasked to defuse the bomb or prevent it from being planted. Both teams can also win a round by eliminating all players on the opposing team before the bomb is planted.

A standard game of Counter-Strike is a best of 30 rounds, the winning team being the first to win 16 rounds. The 30 rounds are played in two halves of 15 on each side of the map, with a round time limit of 1 minute 55 seconds, plus 40 seconds after the bomb is planted.

In case both teams draw at the 30th round on 15x15, 6 more rounds are added-on, which constitutes overtime. The overtime ends if a team wins 4 out of 6 rounds. If both teams win 3 rounds in overtime, another overtime of 6 rounds is played, and the process might repeat indefinitely until one team wins it.

There are 7 maps in the map pool that are available to be played competitively at any given time. Maps are removed and added frequently for updates and revamps, as to not make the game stale. Matches are normally played as a 'bo3' (Best of 3) maps, with less important matches played in a 'bo1' fashion and finals often played as 'bo5's.

# Preparation

In [39]:
# installing plotly for rendering in nbviewer, use ! incase not working
pip install plotly --quiet 
# importing libraries
import lxml
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [40]:
# loading the data
results_df = pd.read_csv('results.csv',low_memory=False)
players_df = pd.read_csv('players.csv',low_memory=False)

The first rows of each table are presented below. We have data in 2 tables that store data related to:

 - <b>results_df:</b>   &nbsp;  map scores and team rankings

 - <b>players_df:</b> &nbsp;   individual performances of players in each map.
 
Values stored in 'event_id' and 'match_id' columns are unique for each match and event and shared between dataframes, so these columns can be used as keys to merge data between dataframes.

It is necessary to note that the rows in 'results_df' stores data for each map played in a match, while the rows in 'players_df' stores data for the entire match.

In [41]:
results_df.head()

,date,team_1,team_2,_map,result_1,result_2,map_winner,starting_ct,ct_1,t_2,t_1,ct_2,event_id,match_id,rank_1,rank_2,map_wins_1,map_wins_2,match_winner
0,18-03-2020,Recon 5,TeamOne,Dust2,0.0,16.0,2.0,2.0,0.0,1.0,0.0,15.0,5151.0,2340454.0,62.0,63.0,0.0,2.0,2.0
1,18-03-2020,Recon 5,TeamOne,Inferno,13.0,16.0,2.0,2.0,8.0,6.0,5.0,10.0,5151.0,2340454.0,62.0,63.0,0.0,2.0,2.0
2,18-03-2020,New England Whalers,Station7,Inferno,12.0,16.0,2.0,1.0,9.0,6.0,3.0,10.0,5243.0,2340461.0,140.0,118.0,12.0,16.0,2.0
3,18-03-2020,Rugratz,Bad News Bears,Inferno,7.0,16.0,2.0,2.0,0.0,8.0,7.0,8.0,5151.0,2340453.0,61.0,38.0,0.0,2.0,2.0
4,18-03-2020,Rugratz,Bad News Bears,Vertigo,8.0,16.0,2.0,2.0,4.0,5.0,4.0,11.0,5151.0,2340453.0,61.0,38.0,0.0,2.0,2.0


In [42]:
players_df.head()

,date,player_name,team,opponent,country,player_id,match_id,event_id,event_name,best_of,...,m3_kddiff_ct,m3_adr_ct,m3_kast_ct,m3_rating_ct,m3_kills_t,m3_deaths_t,m3_kddiff_t,m3_adr_t,m3_kast_t,m3_rating_t
0,26-02-2020,Brehze,Evil Geniuses,Liquid,United States,9136.0,2339385.0,4901.0,IEM Katowice 2020,3.0,...,-1.0,72.5,80.0,0.93,7.0,9.0,-2.0,70.4,63.6,0.89
1,26-02-2020,CeRq,Evil Geniuses,Liquid,Bulgaria,11219.0,2339385.0,4901.0,IEM Katowice 2020,3.0,...,3.0,79.5,53.3,1.12,4.0,8.0,-4.0,40.7,54.5,0.53
2,26-02-2020,EliGE,Liquid,Evil Geniuses,United States,8738.0,2339385.0,4901.0,IEM Katowice 2020,3.0,...,1.0,81.5,63.6,1.03,9.0,9.0,0.0,87.9,73.3,1.05
3,26-02-2020,Ethan,Evil Geniuses,Liquid,United States,10671.0,2339385.0,4901.0,IEM Katowice 2020,3.0,...,0.0,67.2,66.7,0.97,1.0,9.0,-8.0,14.8,45.5,0.31
4,26-02-2020,NAF,Liquid,Evil Geniuses,Canada,8520.0,2339385.0,4901.0,IEM Katowice 2020,3.0,...,-1.0,72.9,81.8,0.96,8.0,7.0,1.0,56.3,80.0,0.99


The data collected has data from all professional CS:GO matches, including matches from relatively unknown teams. For that reason, we will be limiting the datasets to the matches played between the top 30 teams in the HLTV rankings.

In [43]:
min_rank = 30
results_df = results_df[(results_df.rank_1<min_rank)&(results_df.rank_2<min_rank)]
players_df = players_df[players_df.match_id.isin(results_df.match_id.unique())]

# Results DataFrame

In [44]:
results_df.head()

,date,team_1,team_2,_map,result_1,result_2,map_winner,starting_ct,ct_1,t_2,t_1,ct_2,event_id,match_id,rank_1,rank_2,map_wins_1,map_wins_2,match_winner
9,17-03-2020,fnatic,BIG,Mirage,12.0,16.0,2.0,1.0,9.0,6.0,3.0,10.0,5226.0,2340397.0,5.0,18.0,1.0,2.0,2.0
21,17-03-2020,fnatic,BIG,Overpass,16.0,7.0,1.0,2.0,4.0,4.0,12.0,3.0,5226.0,2340397.0,5.0,18.0,1.0,2.0,2.0
22,17-03-2020,Natus Vincere,North,Nuke,10.0,16.0,2.0,1.0,7.0,8.0,3.0,8.0,5226.0,2340396.0,1.0,23.0,0.0,2.0,2.0
23,17-03-2020,Natus Vincere,North,Dust2,10.0,16.0,2.0,1.0,5.0,10.0,5.0,6.0,5226.0,2340396.0,1.0,23.0,0.0,2.0,2.0
24,17-03-2020,forZe,Complexity,Train,9.0,16.0,2.0,2.0,3.0,7.0,6.0,9.0,5226.0,2340279.0,24.0,17.0,2.0,1.0,1.0


## Distribution of scores (final map result)

In [45]:
winner_1 = results_df[results_df.result_1>=results_df.result_2].result_1.values
loser_1  = results_df[results_df.result_1>=results_df.result_2].result_2.values

winner_2 = results_df[results_df.result_1<results_df.result_2].result_2.values
loser_2  = results_df[results_df.result_1<results_df.result_2].result_1.values

winner = np.concatenate((winner_1,winner_2))
loser = np.concatenate((loser_1,loser_2))

In [46]:
scores_df = pd.DataFrame(np.vstack((winner,loser)).T,columns=['winner','loser'])
scores_df

,winner,loser
0,16.0,7.0
1,19.0,17.0
2,16.0,9.0
3,16.0,14.0
4,16.0,8.0
...,...,...
1978,22.0,20.0
1979,16.0,12.0
1980,16.0,14.0
1981,16.0,11.0


In [47]:
# grouping by the winner and loser combination (e.g., 16-14, 16-13, etc)
gb = scores_df.groupby(by=['winner','loser'])['winner'].count()/scores_df.shape[0]

In [48]:
# This is done to measure the overtime percetange of matches
gb.index.get_level_values(0)!=16

array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

We can see that there are some matches where we have draw. For simplicity we will ignore these matches and count into overtime matches.

In [49]:
#overtime percentage
overtime_percentage = str(round(gb[gb.index.get_level_values(0)!=16].sum()*100,1))+'%'
print("{} of the matches go to overtime".format(overtime_percentage))

9.9% of the matches go to overtime


In [50]:
# converting to percentage for plotting
gb = round(gb[gb>10**-3]*100,1)
gb

winner  loser
15.0    15.0      0.4
16.0    0.0       0.4
        1.0       0.8
        2.0       1.9
        3.0       3.0
        4.0       3.6
        5.0       4.2
        6.0       5.0
        7.0       6.8
        8.0       8.0
        9.0       8.6
        10.0      7.8
        11.0      9.6
        12.0     10.1
        13.0      9.8
        14.0     10.6
19.0    15.0      1.4
        16.0      2.0
        17.0      2.8
22.0    18.0      0.2
        19.0      0.9
        20.0      1.4
25.0    21.0      0.1
        22.0      0.2
        23.0      0.3
28.0    25.0      0.2
Name: winner, dtype: float64

In [51]:
# defining the x-axis labels
index_plot = np.array(gb.index.get_level_values(0).astype('str'))+'-'+np.array(
    gb.index.get_level_values(1).astype('str'))


fig = go.Figure()
fig.add_trace(go.Scatter(x=index_plot,y=gb.values, name='results'))
fig.update_layout(xaxis_type='category',title='Scores distribution (final map result)',xaxis_title='Score',yaxis_title='Percentage of matches (%)')
fig.show()

We can see that on regular time, the most common score is 16-14 (achieved in 10.6% of the matches) and the rarest score is 16-0 (achieved in only 0.4% of the matches), with intermediate scores falling somewhere in between. 

9.9% of the matches go to overtime.

The results may differ if we consider matches played by non top-tier teams.

## Most CT sided map

There has long been a dispute in CS:GO to determine the most CT-sided maps, and always present discussion if having a heavily one-sided map is a desirable outcome. Here we determine this characteristic by computing the average scores obtained in each side of the map and then comparing both sides.

In [15]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1983 entries, 9 to 13148
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          1983 non-null   object 
 1   team_1        1983 non-null   object 
 2   team_2        1983 non-null   object 
 3   _map          1983 non-null   object 
 4   result_1      1983 non-null   float64
 5   result_2      1983 non-null   float64
 6   map_winner    1983 non-null   float64
 7   starting_ct   1983 non-null   float64
 8   ct_1          1983 non-null   float64
 9   t_2           1983 non-null   float64
 10  t_1           1983 non-null   float64
 11  ct_2          1983 non-null   float64
 12  event_id      1983 non-null   float64
 13  match_id      1983 non-null   float64
 14  rank_1        1983 non-null   float64
 15  rank_2        1983 non-null   float64
 16  map_wins_1    1983 non-null   float64
 17  map_wins_2    1983 non-null   float64
 18  match_winner  1983 non-null

In [16]:
# converting the datatype of date column to datetime which is originally of object type as we will use to sort the data further.
results_df['date']=pd.to_datetime(results_df['date'],dayfirst=True)

In [18]:
# extracting the rounds won by each team on each map on ct side
ct_1 = results_df[['date','_map','ct_1']].rename(columns={'ct_1':'ct'})
ct_2 = results_df[['date','_map','ct_2']].rename(columns={'ct_2':'ct'})
ct = pd.concat((ct_1,ct_2))
ct

,date,_map,ct
9,2020-03-17,Mirage,9.0
21,2020-03-17,Overpass,4.0
22,2020-03-17,Nuke,7.0
23,2020-03-17,Dust2,5.0
24,2020-03-17,Train,3.0
...,...,...,...
13078,2019-01-18,Nuke,12.0
13106,2019-01-17,Nuke,7.0
13111,2019-01-17,Train,8.0
13126,2019-01-16,Dust2,6.0


In [19]:
# extracting the rounds won by each team on each map on t side
t_1 = results_df[['date','_map','t_1']].rename(columns={'t_1':'t'})
t_2 = results_df[['date','_map','t_2']].rename(columns={'t_2':'t'})
t = pd.concat((t_1,t_2))

In [20]:
t = t.sort_values('date')
ct = ct.sort_values('date')

In [21]:
maps = ['Dust2','Inferno','Mirage','Nuke','Overpass','Train','Vertigo']

In [25]:
series_t, series_ct, how_ct, how_t= {},{},{},{}
for i, key in enumerate(maps):
    t_map = t[t._map == maps[i]]
    ct_map = ct[ct._map == maps[i]]
    
    # rolling sum to lower the effect of variance
    y_t = t_map.t.rolling(min_periods = 20, window= 200, center=True).sum().values
    y_ct = ct_map.ct.rolling(min_periods = 20, window= 200, center=True).sum().values
    
    series_t[key] = pd.Series(data=y_t,index=t_map.date)
    series_ct[key] = pd.Series(data=y_ct,index=ct_map.date)
    
    # percentage of rounds won on ct-side
    how_ct[key] = series_ct[key]/(series_ct[key]+series_t[key])//0.001/10
    how_t[key] = series_t[key]/(series_ct[key]+series_t[key])//0.001/10

In [30]:
def add_trace_ct(_map,n,m):
    fig.add_trace(go.Scatter(x=how_ct[_map].index, y=how_ct[_map].values, name=_map),row=n,col=m)

# here n,m are positions for multiple plots

In [31]:
fig=make_subplots(rows=1,cols=1)
for _map in maps:
    add_trace(_map,1,1)
fig.add_trace(go.Scatter(x=['2019-01-01', '2020-04-12'], y=[50,50],
                         mode='lines',line=dict(color='grey'),showlegend=False))
fig.update_layout(title='Distribution of rounds between CT and T sides',
                  yaxis_title='Percentage of round won on the CT-side (%)')
fig.show()

There are long stretches without data for a map in the graph. This happens because maps are added and removed from the map pool constantly.

<b>Nuke</b> and <b>Train</b> oscilatte as being the most CT-sided maps, having around 57% of the rounds won on the CT-side, while
<b>Dust2</b> is historically the most T-sided maps.

<b>Mirage</b> in early months of 2019 seems to be a CT-sided map, but after that it becomes neutral. <b>Overpass</b> has a constant rate of 55 % rounds won on CT-side.

## Maps played per period

In [32]:
print('Total number of matches played on the map:')
results_df.groupby('_map').date.count()

Total number of matches played on the map:


_map
Cache        36
Dust2       350
Inferno     332
Mirage      347
Nuke        309
Overpass    258
Train       252
Vertigo      99
Name: date, dtype: int64

About the maps:
 - <b>Mirage</b>, <b>Inferno</b> and <b>Dust2</b> are the maps from which we have the most data available. They are also the maps present on the map pool for the longest time;

 - <b>Nuke</b> and <b>Overpass</b> have slightly less data which should be because of teams' unfamiliarity with the map;

 - <b>Vertigo</b> has limited data available as it was the most recently added map in the map pool;

 - <b>Cache</b> was removed from the competitive pool in early months of 2019.

In CS:GO, the most reputable tournaments are the Majors. These tournaments are normally played twice a year and have a prize pool of $1,000,000. More information about the subject can be seen here: https://liquipedia.net/counterstrike/Majors

For the next step, we are going to discretize the 'date' column in a dataframe into a 'time_period' column. This new column will refer to the most recently played major tournament.

As an example, following this binning technique, we are currently on the Berlin 2019 period, as that was the most recently played tournament in the dataset available.

In [33]:
majors = [{'tournament':'01. London 2018','start_date':'2018-09-20'},
          {'tournament':'02. Katowice 2019','start_date':'2019-02-28'},
          {'tournament':'03. Berlin 2019','start_date':'2019-09-05'}]

In [38]:
# function to create new column 'time_period' which assigns each match their respective major name.
def create_col_time_period(df):
    df['time_period'] = ''
    
    for major_start in majors:
        df.loc[(df['date']>=major_start['start_date']),'time_period'] = major_start['tournament']
    
    return df

In [39]:
# adding new column `time_period` in both the tables.
results_df = create_col_time_period(results_df)
players_df = players_df.merge(results_df[['match_id','time_period']],'left',on='match_id')

In [40]:
players_df.head()

,date,player_name,team,opponent,country,player_id,match_id,event_id,event_name,best_of,...,m3_adr_ct,m3_kast_ct,m3_rating_ct,m3_kills_t,m3_deaths_t,m3_kddiff_t,m3_adr_t,m3_kast_t,m3_rating_t,time_period
0,26-02-2020,Brehze,Evil Geniuses,Liquid,United States,9136.0,2339385.0,4901.0,IEM Katowice 2020,3.0,...,72.5,80.0,0.93,7.0,9.0,-2.0,70.4,63.6,0.89,03. Berlin 2019
1,26-02-2020,Brehze,Evil Geniuses,Liquid,United States,9136.0,2339385.0,4901.0,IEM Katowice 2020,3.0,...,72.5,80.0,0.93,7.0,9.0,-2.0,70.4,63.6,0.89,03. Berlin 2019
2,26-02-2020,Brehze,Evil Geniuses,Liquid,United States,9136.0,2339385.0,4901.0,IEM Katowice 2020,3.0,...,72.5,80.0,0.93,7.0,9.0,-2.0,70.4,63.6,0.89,03. Berlin 2019
3,26-02-2020,CeRq,Evil Geniuses,Liquid,Bulgaria,11219.0,2339385.0,4901.0,IEM Katowice 2020,3.0,...,79.5,53.3,1.12,4.0,8.0,-4.0,40.7,54.5,0.53,03. Berlin 2019
4,26-02-2020,CeRq,Evil Geniuses,Liquid,Bulgaria,11219.0,2339385.0,4901.0,IEM Katowice 2020,3.0,...,79.5,53.3,1.12,4.0,8.0,-4.0,40.7,54.5,0.53,03. Berlin 2019


In [41]:
results_df_team_1 = results_df[['time_period','team_1','_map','ct_1','t_2','ct_2','t_1']
                      ].rename(columns={'team_1':'team'})
results_df_team_2 = results_df[['time_period','team_2','_map','ct_1','t_2','ct_2','t_1']
                      ].rename(columns={'team_2':'team'})
results_df_teams = pd.concat((results_df_team_1,results_df_team_2))[['time_period','team','_map']]
# we're creating a seperate df which basically includes the major name, team and which maps they played in that major

In [49]:
results_df_teams

,time_period,team,_map
9,03. Berlin 2019,fnatic,Mirage
21,03. Berlin 2019,fnatic,Overpass
22,03. Berlin 2019,Natus Vincere,Nuke
23,03. Berlin 2019,Natus Vincere,Dust2
24,03. Berlin 2019,forZe,Train
...,...,...,...
13078,01. London 2018,North,Nuke
13106,01. London 2018,mousesports,Nuke
13111,01. London 2018,mousesports,Train
13126,01. London 2018,OpTic,Dust2


In [43]:
# contains the no of times the map has been played in each major.
gb = results_df_teams.groupby(['time_period','_map']).team.count()
gb

time_period        _map    
01. London 2018    Cache        32
                   Dust2        58
                   Inferno      58
                   Mirage       82
                   Nuke         34
                   Overpass     48
                   Train        48
02. Katowice 2019  Cache        40
                   Dust2       326
                   Inferno     290
                   Mirage      312
                   Nuke        254
                   Overpass    268
                   Train       216
                   Vertigo      60
03. Berlin 2019    Dust2       316
                   Inferno     316
                   Mirage      300
                   Nuke        330
                   Overpass    200
                   Train       240
                   Vertigo     138
Name: team, dtype: int64

In [44]:
# creating a new df which contains the percentage of map played in each major.
gb_text = round(gb*100/gb.groupby('time_period').sum(),1).reset_index().rename(columns={'team':'percentage'})
gb_text.percentage = gb_text.percentage.astype(str)+'%'
gb = gb.reset_index()

In [45]:
fig = go.Figure()
for _map in maps:
    fig.add_bar(name=_map,x=gb[gb._map==_map].time_period,y=gb[gb._map==_map].team,
                text=gb_text[gb_text._map==_map].percentage,textposition='inside')

fig.update_layout(barmode='stack',legend=dict(traceorder='normal'),yaxis_title='Number of maps played',font=dict(size=10))
fig.show()

<b>Vertigo</b>, as the newest and most unconventional map, is also the most unpopular map, probably due to the many changes it has had in its short competitive term.

<b>Nuke</b>, is the most played map in the <b>Berlin 2019</b> major while in the previous major <b>Katowice 2019</b> it was 5th most played.

## Best teams on each map

In this section, let's search for the best teams in each map. The victory percentage for the teams on each map is readily available in many e-sports sites like hltv.org, so it would be pointless to present the information the same way. For this reason, instead of pointing out the percentages, we are going to rank the best teams on each map and each side (CT and T) for every time period (major).

In [46]:
results_df_team_1_ct = results_df_team_1.rename(columns={'ct_1':'ct_team','t_2':'t_opponent'}).drop(columns=['ct_2','t_1'])
results_df_team_2_ct = results_df_team_2.rename(columns={'ct_2':'ct_team','t_1':'t_opponent'}).drop(columns=['ct_1','t_2'])
results_df_ct = pd.concat((results_df_team_1_ct,results_df_team_2_ct),sort=True)

results_df_team_1_t = results_df_team_1.rename(columns={'t_1':'t_team','ct_2':'ct_opponent'}).drop(columns=['ct_1','t_2'])
results_df_team_2_t = results_df_team_2.rename(columns={'t_2':'t_team','ct_1':'ct_opponent'}).drop(columns=['ct_2','t_1'])
results_df_t = pd.concat((results_df_team_1_t,results_df_team_2_t),sort=True)

In [47]:
results_df_ct['side_diff'] = results_df_ct['ct_team']-results_df_ct['t_opponent']
results_df_ct['side_sum'] = results_df_ct['ct_team']+results_df_ct['t_opponent']

results_df_t['side_diff'] = results_df_t['t_team']-results_df_t['ct_opponent']
results_df_t['side_sum']  = results_df_t['t_team'] +results_df_t['ct_opponent']

results_df_ct.head()

,_map,ct_team,t_opponent,team,time_period,side_diff,side_sum
9,Mirage,9.0,6.0,fnatic,03. Berlin 2019,3.0,15.0
21,Overpass,4.0,4.0,fnatic,03. Berlin 2019,0.0,8.0
22,Nuke,7.0,8.0,Natus Vincere,03. Berlin 2019,-1.0,15.0
23,Dust2,5.0,10.0,Natus Vincere,03. Berlin 2019,-5.0,15.0
24,Train,3.0,7.0,forZe,03. Berlin 2019,-4.0,10.0


In [49]:
gb = results_df_ct.groupby(['time_period','_map','team'])['side_diff','side_sum'].sum()
gb['side_diff_per_game'] = gb['side_diff']/(gb['side_sum']/15)
gb

C:\Users\Dharmendra Dudi\AppData\Local\Temp\ipykernel_7572\672866683.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



side_diff  side_sum  side_diff_per_game
time_period     _map    team                                                
01. London 2018 Cache   3DMAX             -7.0       7.0          -15.000000
                        AVANGAR            4.0      30.0            2.000000
                        BIG               24.0      60.0            6.000000
                        Cloud9            14.0      58.0            3.620690
                        FURIA             -8.0      22.0           -5.454545
...                                        ...       ...                 ...
03. Berlin 2019 Vertigo Virtus.pro       -28.0     170.0           -2.470588
                        Vitality          -5.0      75.0           -1.000000
                        fnatic             2.0      60.0            0.500000
                        forZe              0.0      60.0            0.000000
                        mousesports      -27.0     147.0           -2.755102

[674 rows x 3 columns]

In [50]:
def groupby_time_map_team(results_df_side):
    gb = results_df_side.groupby(['time_period','_map','team'])['side_diff','side_sum'].sum()
    gb['side_diff_per_game'] = gb['side_diff']/(gb['side_sum']/15)
    gb = gb.sort_values(['time_period','_map','side_diff_per_game'],ascending=[1,1,0])

    for major in majors:
        col = major['tournament']
        _filter = (gb.side_sum > gb.loc[col].side_sum.mean()*3/4)
        gb.loc[col] = gb.loc[_filter][gb.loc[_filter].index.get_level_values(0)==col]

    gb.dropna(inplace=True)    

    return gb

In [51]:
gb_ct = groupby_time_map_team(results_df_ct)
gb_t = groupby_time_map_team(results_df_t)

C:\Users\Dharmendra Dudi\AppData\Local\Temp\ipykernel_7572\96439937.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

C:\Users\Dharmendra Dudi\AppData\Local\Temp\ipykernel_7572\96439937.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [52]:
def plot_ranking_teams_sides(gb):
    rankings_teams = {}
    for _map in maps:
        rankings_teams[_map] = pd.DataFrame(index=range(1,6),)
        rankings_teams[_map].index.name = 'ranking'
        rankings_teams[_map].style.set_caption(_map)

        for major in majors:
            col = major['tournament']
            try:
                rankings_teams[_map][col] = gb.loc[col,_map]['side_diff_per_game'][:5].index
            except:
                pass
        print('\n'+_map+':')
        display(rankings_teams[_map])

In [53]:
print('T-side Rankings:\n')
plot_ranking_teams_sides(gb_t)

T-side Rankings:


Dust2:


,01. London 2018,02. Katowice 2019,03. Berlin 2019
ranking,,,
1,Windigo,mousesports,mousesports
2,North,G2,Natus Vincere
3,Liquid,CR4ZY,Astralis
4,Valiance,Windigo,FaZe
5,forZe,Liquid,G2



Inferno:


,01. London 2018,02. Katowice 2019,03. Berlin 2019
ranking,,,
1,Vitality,Sprout,100 Thieves
2,Cloud9,Astralis,forZe
3,Astralis,Liquid,NiP
4,Valiance,ENCE,mousesports
5,Liquid,Natus Vincere,FURIA



Mirage:


...

,01. London 2018,02. Katowice 2019,03. Berlin 2019
ranking,,,
1,FaZe,Liquid,FURIA
2,Cloud9,Heroic,fnatic
3,forZe,OpTic,Astralis
4,HellRaisers,MIBR,Tricked
5,Vitality,Vitality,NiP



Train:


,01. London 2018,02. Katowice 2019,03. Berlin 2019
ranking,,,
1,FaZe,Natus Vincere,Sprout
2,Valiance,NRG,FURIA
3,Ghost,Valiance,NiP
4,ENCE,fnatic,Astralis
5,BIG,North,mousesports



Vertigo:


,03. Berlin 2019
ranking,
1,GODSENT
2,G2
3,MIBR
4,NiP
5,Heroic


In [73]:
print('CT-side Rankings:\n')
plot_ranking_teams_sides(gb_ct)

CT-side Rankings:


Dust2:


,01. London 2018,02. Katowice 2019,03. Berlin 2019
ranking,,,
1,BIG,OpTic,GODSENT
2,forZe,North,ENCE
3,G2,Vitality,fnatic
4,Liquid,FaZe,North
5,Sprout,Natus Vincere,Natus Vincere



Inferno:


,01. London 2018,02. Katowice 2019,03. Berlin 2019
ranking,,,
1,Astralis,Astralis,Liquid
2,ENCE,mousesports,G2
3,Cloud9,Valiance,Astralis
4,FaZe,AVANGAR,fnatic
5,Renegades,Liquid,Vitality



Mirage:


...

,01. London 2018,02. Katowice 2019,03. Berlin 2019
ranking,,,
1,ENCE,North,Natus Vincere
2,Vitality,Liquid,Sprout
3,Liquid,NiP,mousesports
4,NiP,Astralis,FURIA
5,AVANGAR,Vitality,Tricked



Train:


,01. London 2018,02. Katowice 2019,03. Berlin 2019
ranking,,,
1,Astralis,NRG,AVANGAR
2,Winstrike,Astralis,CR4ZY
3,ENCE,ENCE,mousesports
4,forZe,Natus Vincere,Astralis
5,BIG,FaZe,Natus Vincere



Vertigo:


,03. Berlin 2019
ranking,
1,Astralis
2,G2
3,Heroic
4,Liquid
5,NiP


# Players DataFrame

## Players ranking by map

In [55]:
pd.set_option('display.max_columns', None)
players_df.head()

,date,player_name,team,opponent,country,player_id,match_id,event_id,event_name,best_of,map_1,map_2,map_3,kills,assists,deaths,hs,flash_assists,kast,kddiff,adr,fkdiff,rating,m1_kills,m1_assists,m1_deaths,m1_hs,m1_flash_assists,m1_kast,m1_kddiff,m1_adr,m1_fkdiff,m1_rating,m2_kills,m2_assists,m2_deaths,m2_hs,m2_flash_assists,m2_kast,m2_kddiff,m2_adr,m2_fkdiff,m2_rating,m3_kills,m3_assists,m3_deaths,m3_hs,m3_flash_assists,m3_kast,m3_kddiff,m3_adr,m3_fkdiff,m3_rating,kills_ct,deaths_ct,kddiff_ct,adr_ct,kast_ct,rating_ct,kills_t,deaths_t,kddiff_t,adr_t,kast_t,rating_t,m1_kills_ct,m1_deaths_ct,m1_kddiff_ct,m1_adr_ct,m1_kast_ct,m1_rating_ct,m1_kills_t,m1_deaths_t,m1_kddiff_t,m1_adr_t,m1_kast_t,m1_rating_t,m2_kills_ct,m2_deaths_ct,m2_kddiff_ct,m2_adr_ct,m2_kast_ct,m2_rating_ct,m2_kills_t,m2_deaths_t,m2_kddiff_t,m2_adr_t,m2_kast_t,m2_rating_t,m3_kills_ct,m3_deaths_ct,m3_kddiff_ct,m3_adr_ct,m3_kast_ct,m3_rating_ct,m3_kills_t,m3_deaths_t,m3_kddiff_t,m3_adr_t,m3_kast_t,m3_rating_t,time_period
0,26-02-2020,Brehze,Evil Geniuses,Liquid,United States,9136.0,2339385.0,4901.0,IEM Katowice 2020,3.0,Overpass,Nuke,Inferno,57.0,14.0,61.0,29.0,0.0,71.1,-4.0,79.9,0.0,1.04,11.0,3.0,18.0,5.0,0.0,65.2,-7.0,60.8,-1.0,0.70,30.0,7.0,24.0,16.0,0.0,73.5,6.0,99.2,6.0,1.38,16.0,4.0,19.0,8.0,0.0,73.1,-3.0,71.6,-5.0,0.91,34.0,30.0,4.0,81.6,79.2,1.10,23.0,31.0,-8.0,77.5,60.0,0.97,8.0,10.0,-2.0,76.3,73.3,0.90,3.0,8.0,-5.0,31.9,50.0,0.34,17.0,10.0,7.0,93.7,83.3,1.41,13.0,14.0,-1.0,105.3,62.5,1.35,9.0,10.0,-1.0,72.5,80.0,0.93,7.0,9.0,-2.0,70.4,63.6,0.89,03. Berlin 2019
1,26-02-2020,Brehze,Evil Geniuses,Liquid,United States,9136.0,2339385.0,4901.0,IEM Katowice 2020,3.0,Overpass,Nuke,Inferno,57.0,14.0,61.0,29.0,0.0,71.1,-4.0,79.9,0.0,1.04,11.0,3.0,18.0,5.0,0.0,65.2,-7.0,60.8,-1.0,0.70,30.0,7.0,24.0,16.0,0.0,73.5,6.0,99.2,6.0,1.38,16.0,4.0,19.0,8.0,0.0,73.1,-3.0,71.6,-5.0,0.91,34.0,30.0,4.0,81.6,79.2,1.10,23.0,31.0,-8.0,77.5,60.0,0.97,8.0,10.0,-2.0,76.3,73.3,0.90,3.0,8.0,-5.0,31.9,50.0,0.34,17.0,10.0,7.0,93.7,83.3,1.41,13.0,14.0,-1.0,105.3,62.5,1.35,9.0,10.0,-1.0,72.5,80.0,0.93,7.0,9.0,-2.0,70.4,63.6,0.89,03. Berlin 2019
2,26-02-2020,Brehze,Evil Geniuses,Liquid,United States,9136.0,2339385.0,4901.0,IEM Katowice 2020,3.0,Overpass,Nuke,Inferno,57.0,14.0,61.0,29.0,0.0,71.1,-4.0,79.9,0.0,1.04,11.0,3.0,18.0,5.0,0.0,65.2,-7.0,60.8,-1.0,0.70,30.0,7.0,24.0,16.0,0.0,73.5,6.0,99.2,6.0,1.38,16.0,4.0,19.0,8.0,0.0,73.1,-3.0,71.6,-5.0,0.91,34.0,30.0,4.0,81.6,79.2,1.10,23.0,31.0,-8.0,77.5,60.0,0.97,8.0,10.0,-2.0,76.3,73.3,0.90,3.0,8.0,-5.0,31.9,50.0,0.34,17.0,10.0,7.0,93.7,83.3,1.41,13.0,14.0,-1.0,105.3,62.5,1.35,9.0,10.0,-1.0,72.5,80.0,0.93,7.0,9.0,-2.0,70.4,63.6,0.89,03. Berlin 2019
3,26-02-2020,CeRq,Evil Geniuses,Liquid,Bulgaria,11219.0,2339385.0,4901.0,IEM Katowice 2020,3.0,Overpass,Nuke,Inferno,54.0,10.0,54.0,18.0,4.0,65.1,0.0,71.7,2.0,0.98,11.0,2.0,17.0,4.0,2.0,60.9,-6.0,68.9,-1.0,0.75,26.0,6.0,19.0,8.0,1.0,76.5,7.0,80.1,3.0,1.24,17.0,2.0,18.0,6.0,1.0,53.8,-1.0,63.1,0.0,0.87,37.0,25.0,12.0,77.4,72.9,1.16,17.0,29.0,-12.0,63.9,54.3,0.73,9.0,9.0,0.0,72.3,73.3,0.88,2.0,8.0,-6.0,62.4,37.5,0.50,15.0,6.0,9.0,79.8,88.9,1.45,11.0,13.0,-2.0,80.5,62.5,1.00,13.0,10.0,3.0,79.5,53.3,1.12,4.0,8.0,-4.0,40.7,54.5,0.53,03. Berlin 2019
4,26-02-2020,CeRq,Evil Geniuses,Liquid,Bulgaria,11219.0,2339385.0,4901.0,IEM Katowice 2020,3.0,Overpass,Nuke,Inferno,54.0,10.0,54.0,18.0,4.0,65.1,0.0,71.7,2.0,0.98,11.0,2.0,17.0,4.0,2.0,60.9,-6.0,68.9,-1.0,0.75,26.0,6.0,19.0,8.0,1.0,76.5,7.0,80.1,3.0,1.24,17.0,2.0,18.0,6.0,1.0,53.8,-1.0,63.1,0.0,0.87,37.0,25.0,12.0,77.4,72.9,1.16,17.0,29.0,-12.0,63.9,54.3,0.73,9.0,9.0,0.0,72.3,73.3,0.88,2.0,8.0,-6.0,62.4,37.5,0.50,15.0,6.0,9.0,79.8,88.9,1.45,11.0,13.0,-2.0,80.5,62.5,1.00,13.0,10.0,3.0,79.5,53.3,1.12,4.0,8.0,-4.0,40.7,54.5,0.53,03. Berlin 2019


In [56]:
all_maps_columns = ['date','time_period','country','player_name','team','opponent','player_id',
                    'match_id','event_id','event_name','best_of']
each_map_columns = ['kills','assists','deaths','hs','flash_assists','kast','kddiff','adr','fkdiff','rating']

In [57]:
map1_columns = ['map_1']+['m1_'+ x for x in each_map_columns]
map2_columns = ['map_2']+['m2_'+ x for x in each_map_columns]
map3_columns = ['map_3']+['m3_'+ x for x in each_map_columns]

In [58]:
out_columns = all_maps_columns+['_map']+each_map_columns

players_df_by_map_columns = pd.DataFrame(columns=out_columns)

In [60]:
# Countries that contribute the most to the professional scene by number of matches
players_df.groupby('country')['country'].count().sort_values(ascending=False)[:30]

country
Denmark                   3150
Sweden                    2091
France                    1773
Russia                    1400
United States             1372
Brazil                    1276
Finland                    921
Germany                    851
Canada                     635
Kazakhstan                 610
Poland                     535
Serbia                     531
Australia                  525
Bulgaria                   474
Ukraine                    376
United Kingdom             325
Slovakia                   319
Norway                     287
Turkey                     284
Estonia                    180
Netherlands                174
Lithuania                  169
Bosnia and Herzegovina     162
Uzbekistan                 149
New Zealand                145
China                      132
Latvia                      82
Jordan                      55
Belgium                     48
Indonesia                   36
Name: country, dtype: int64

In [61]:
curr_map = {}
curr_map[0] = players_df[(all_maps_columns+map1_columns)]
curr_map[1] = players_df[(all_maps_columns+map2_columns)]
curr_map[2] = players_df[(all_maps_columns+map3_columns)]

curr_map[0].columns = out_columns
curr_map[1].columns = out_columns
curr_map[2].columns = out_columns

all_maps = pd.concat((pd.concat((curr_map[0],curr_map[1])), curr_map[2]))

In [62]:
gb2 = all_maps.groupby(['time_period','player_id','_map'])
threshold_maps_played = 7
all_maps2 = gb2.filter(lambda x:x.player_name.count()>threshold_maps_played)
all_maps2.head()

,date,time_period,country,player_name,team,opponent,player_id,match_id,event_id,event_name,best_of,_map,kills,assists,deaths,hs,flash_assists,kast,kddiff,adr,fkdiff,rating
0,26-02-2020,03. Berlin 2019,United States,Brehze,Evil Geniuses,Liquid,9136.0,2339385.0,4901.0,IEM Katowice 2020,3.0,Overpass,11.0,3.0,18.0,5.0,0.0,65.2,-7.0,60.8,-1.0,0.70
1,26-02-2020,03. Berlin 2019,United States,Brehze,Evil Geniuses,Liquid,9136.0,2339385.0,4901.0,IEM Katowice 2020,3.0,Overpass,11.0,3.0,18.0,5.0,0.0,65.2,-7.0,60.8,-1.0,0.70
2,26-02-2020,03. Berlin 2019,United States,Brehze,Evil Geniuses,Liquid,9136.0,2339385.0,4901.0,IEM Katowice 2020,3.0,Overpass,11.0,3.0,18.0,5.0,0.0,65.2,-7.0,60.8,-1.0,0.70
3,26-02-2020,03. Berlin 2019,Bulgaria,CeRq,Evil Geniuses,Liquid,11219.0,2339385.0,4901.0,IEM Katowice 2020,3.0,Overpass,11.0,2.0,17.0,4.0,2.0,60.9,-6.0,68.9,-1.0,0.75
4,26-02-2020,03. Berlin 2019,Bulgaria,CeRq,Evil Geniuses,Liquid,11219.0,2339385.0,4901.0,IEM Katowice 2020,3.0,Overpass,11.0,2.0,17.0,4.0,2.0,60.9,-6.0,68.9,-1.0,0.75


In [63]:
gb = all_maps2.groupby(['time_period','_map','player_name'],sort=False)['rating','kddiff'].mean()
rankings = gb.sort_values(['time_period','_map','rating'],ascending=[1,1,0])

C:\Users\Dharmendra Dudi\AppData\Local\Temp\ipykernel_7572\2869439418.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [64]:
rankings_players = {}
for _map in maps:
    rankings_players[_map] = pd.DataFrame(index=range(1,21))
    rankings_players[_map].index.name = 'ranking'
    
    for major in majors:
        col = major['tournament']
        try:
            rankings_players[_map][col] = rankings.loc[col,_map].rating[:20].index
        except:
            pass

C:\Users\Dharmendra Dudi\AppData\Local\Temp\ipykernel_7572\20905936.py:9: PerformanceWarning:

indexing past lexsort depth may impact performance.

C:\Users\Dharmendra Dudi\AppData\Local\Temp\ipykernel_7572\20905936.py:9: PerformanceWarning:

indexing past lexsort depth may impact performance.

C:\Users\Dharmendra Dudi\AppData\Local\Temp\ipykernel_7572\20905936.py:9: PerformanceWarning:

indexing past lexsort depth may impact performance.

C:\Users\Dharmendra Dudi\AppData\Local\Temp\ipykernel_7572\20905936.py:9: PerformanceWarning:

indexing past lexsort depth may impact performance.

C:\Users\Dharmendra Dudi\AppData\Local\Temp\ipykernel_7572\20905936.py:9: PerformanceWarning:

indexing past lexsort depth may impact performance.

C:\Users\Dharmendra Dudi\AppData\Local\Temp\ipykernel_7572\20905936.py:9: PerformanceWarning:

indexing past lexsort depth may impact performance.

C:\Users\Dharmendra Dudi\AppData\Local\Temp\ipykernel_7572\20905936.py:9: PerformanceWarning:

indexing past lex

Best players on each map according to their average Rating 2.0:

In [65]:
for _map in maps:
    print('\n'+_map+':')
    display(rankings_players[_map])


Dust2:


,01. London 2018,02. Katowice 2019,03. Berlin 2019
ranking,,,
1,Twistzz,ZywOo,syrsoN
2,EliGE,electronic,Maden
3,tabseN,woxic,tabseN
4,XANTARES,s1mple,hampus
5,s1mple,ropz,device
6,electronic,rain,k1to
7,NAF,NiKo,KRIMZ
8,Lucky,EliGE,ropz
9,shox,kennyS,dennis



Inferno:


,01. London 2018,02. Katowice 2019,03. Berlin 2019
ranking,,,
1,autimatic,poizon,BnTeT
2,gla1ve,syrsoN,ZywOo
3,kioShiMa,qikert,dennis
4,flusha,Jame,EliGE
5,device,v1c7oR,jks
6,Magisk,SHiPZ,Maikelele
7,Xyp9x,XANTARES,jkaem
8,Kjaerbye,electronic,SHiPZ
9,woxic,s1mple,Plopski



Mirage:


...

,02. Katowice 2019,03. Berlin 2019
ranking,,
1,TeSeS,yuurih
2,NiKo,roeJ
3,REZ,rain
4,ZywOo,ropz
5,EliGE,acoR
6,Jame,coldzera
7,blameF,Brollan
8,s1mple,Magisk
9,Boombl4,KSCERATO



Train:


,01. London 2018,02. Katowice 2019,03. Berlin 2019
ranking,,,
1,GuardiaN,s1mple,s1mple
2,Boombl4,electronic,electronic
3,NiKo,CeRq,Brollan
4,Jame,nexa,mouz
5,Aerial,Xyp9x,almazer
6,tiziaN,aizy,xsepower
7,huNter-,Brollan,HEN1
8,Kvik,Brehze,innocent
9,tabseN,NiKo,nawwk



Vertigo:


,02. Katowice 2019,03. Berlin 2019
ranking,,
1,k0nfig,Brollan
2,jkaem,STYKO
3,EliGE,huNter-
4,device,zehN
5,huNter-,jks
6,nexa,device
7,Xyp9x,Golden
8,MSL,Liazz
9,niko,STOMP


## Players ranking all maps

In [66]:
ranking_players_df = pd.DataFrame()
ranking_players_df['player'] = players_df.player_name.unique()
ranking_players_df.set_index('player',inplace=True)

for major in majors:
    col = major['tournament']
    ranking_players_df[col] = 0
    
for _map in maps:
    for col in rankings_players[_map].columns:
        for i in range(1,21):
            ranking_players_df.loc[rankings_players[_map][col][i],col] += 21-i
            
rankings_players_again = {}

rankings_players_again = pd.DataFrame(index=range(1,21))
rankings_players_again.index.name = 'ranking'

for major in majors:
    col = major['tournament']
    rankings_players_again[col] = ranking_players_df[col].sort_values(ascending=False)[:20].index

Below it is shown the player rankings if it was only determined by the rating 2.0 across all maps. It can be considered as the ranking of most versatile fraggers. 

In [67]:
rankings_players_again

,01. London 2018,02. Katowice 2019,03. Berlin 2019
ranking,,,
1,Jame,s1mple,ZywOo
2,XANTARES,electronic,device
3,huNter-,ZywOo,s1mple
4,tabseN,EliGE,Brollan
5,autimatic,Jame,ropz
6,kioShiMa,device,KSCERATO
7,woxic,nexa,huNter-
8,GuardiaN,NiKo,tabseN
9,Twistzz,huNter-,syrsoN


## K/D Graph

In [68]:
results_df_rank_part_1 = results_df[['match_id','team_1','rank_1']].rename(columns={'team_1':'team','rank_1':'team_rank'})
results_df_rank_part_2 = results_df[['match_id','team_2','rank_2']].rename(columns={'team_2':'team','rank_2':'team_rank'})
results_df_rank = pd.concat((results_df_rank_part_1,results_df_rank_part_2))

In [69]:
all_maps3 = all_maps2.merge(results_df_rank,'left',on=['match_id','team'])

In [70]:
players_series = all_maps3.groupby('player_name').team_rank.min()
players_list = list(players_series[players_series<=3].index)

In [71]:
gb = all_maps3.groupby(['time_period','player_name','country'])['kills','deaths','team_rank']
gb = gb.mean()[gb.count()['kills']>100]
gb = gb[gb.index.get_level_values(1).isin(players_list)]

gb['kills'] = gb['kills'].round(1)
gb['deaths'] = gb['deaths'].round(1)
gb['team_rank'] = gb['team_rank'].round(0).astype('int')
gb.reset_index(inplace=True)

C:\Users\Dharmendra Dudi\AppData\Local\Temp\ipykernel_7572\308125854.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [72]:
gb['region'] = ''
gb.loc[(gb['country']=='Ukraine') | (gb['country']=='Russia') | (gb['country']=='Kazakhstan'),'region'] = 'CIS'
gb.loc[(gb['country']=='Brazil'),'region'] = 'Brazil'
gb.loc[(gb['country']=='France') | (gb['country']=='Belgium'),'region'] = 'France/Belgium'
gb.loc[(gb['country']=='United States') | (gb['country']=='Canada'),'region'] = 'North America'
gb.loc[(gb['country']=='Denmark'),'region'] = 'Denmark'
gb.loc[(gb['country']=='Sweden'),'region'] = 'Sweden'
gb.loc[(gb['country']=='Poland'),'region'] = 'Poland'
gb.loc[gb.country.isin(['Netherlands','Slovakia','Bosnia and Herzegovina',
                        'Norway','Czech Republic','Spain','Estonia','United Kingdom','Portugal','Turkey','Bulgaria', 'Finland']),'region'] = 'Rest of Europe'

gb = gb.sort_values(['time_period','region'])

In [73]:
gb['size'] = (100/(gb['team_rank']+2)).round(1)

In [74]:
fig = px.scatter(gb, x="deaths", y="kills", animation_frame="time_period", animation_group="player_name",
           size="size", color="region", hover_name="player_name", hover_data=["team_rank"],
                 range_x=[14,24],range_y=[14,24]
                )
#.for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 2000
fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 1300
fig.update_layout(xaxis_title='Deaths', yaxis_title = 'Kills')
fig.add_shape(type="line", x0=14, y0=14, x1=24, y1=24, line=dict(width=4, dash="dot"))
fig.update_shapes(dict(xref='x', yref='y'))
fig.show()